In [214]:
from __future__ import absolute_import, division, print_function

import tensorflow as tf
import numpy as np

print(tf.__version__)
from sklearn.model_selection import train_test_split
from sklearn.datasets.california_housing import fetch_california_housing

1.11.0-rc1


In [215]:
cal_housing = fetch_california_housing()
# split 80/20 train-test
X_train, X_test, y_train, y_test = train_test_split(cal_housing.data,
                                                    cal_housing.target,
                                                        test_size=0.2,
                                                        random_state=1)
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

X_train = X_train / X_train.max(axis=0)
X_test = X_test / X_test.max(axis=0)
y_train = y_train / y_train.max(axis=0)
y_test = y_test / y_test.max(axis=0)

(16512, 8)
(4128, 8)
(16512,)
(4128,)


In [216]:
def convert(x):
    return tf.convert_to_tensor(x, dtype=tf.float32)

y_test = convert(y_test)
X_test = convert(X_test)

In [217]:
# Parameters
alpha = 0.01
training_epochs = 25
display_step = 1
n_hidden = 5

# tf Graph input
X = tf.placeholder(tf.float32, shape=[None, 8])
Y = tf.placeholder(tf.float32, shape=[None])

In [218]:
# Store layers weight & bias
weights = {
    'h1': tf.Variable(tf.truncated_normal([8, 4], stddev=0.03)),
    'out': tf.Variable(tf.truncated_normal([4, 1], stddev=0.03))
}
biases = {
    'b1': tf.Variable(tf.truncated_normal([4])),
    'out': tf.Variable(tf.truncated_normal([1]))
}

In [225]:

# Create model
def neural_net(x):
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    layer_1 = tf.sigmoid(layer_1)
    # Output fully connected layer with a neuron for each class
    out_layer = tf.matmul(layer_1, weights['out']) + biases['out']
    out_layer = tf.sigmoid(out_layer)
    return out_layer

In [226]:
# Construct model
pred = neural_net(X)

# Define loss and optimizer
mse = tf.reduce_mean(tf.square(pred - Y))  # When this doesn't?
optimizer = tf.train.GradientDescentOptimizer(learning_rate=alpha).minimize(mse)

init = tf.global_variables_initializer()

In [221]:
with tf.Session() as sess:
    sess.run(init)
    # Training cycle
    for epoch in range(training_epochs):
        avg_cost = 0.
        # Loop over all batches
        for i in range(100):
            # Run optimization op (backprop) and cost op (to get loss value)
            _, msev = sess.run([optimizer, mse], feed_dict={X: X_train, Y: y_train})
        # Display logs per epoch step
        if epoch % display_step == 0:
            print("Epoch:", '%04d' % (epoch+1), "mse=", \
                "{:.9f}".format(msev))

Epoch: 0001 mse= 0.084464736
Epoch: 0002 mse= 0.068458036
Epoch: 0003 mse= 0.060536671
Epoch: 0004 mse= 0.056786817
Epoch: 0005 mse= 0.055028331
Epoch: 0006 mse= 0.054200422
Epoch: 0007 mse= 0.053807683
Epoch: 0008 mse= 0.053620007
Epoch: 0009 mse= 0.053529758
Epoch: 0010 mse= 0.053486161
Epoch: 0011 mse= 0.053465024
Epoch: 0012 mse= 0.053454753
Epoch: 0013 mse= 0.053449750
Epoch: 0014 mse= 0.053447310
Epoch: 0015 mse= 0.053446118
Epoch: 0016 mse= 0.053445540
Epoch: 0017 mse= 0.053445254
Epoch: 0018 mse= 0.053445116
Epoch: 0019 mse= 0.053445052
Epoch: 0020 mse= 0.053445011
Epoch: 0021 mse= 0.053445000
Epoch: 0022 mse= 0.053444989
Epoch: 0023 mse= 0.053444985
Epoch: 0024 mse= 0.053444985
Epoch: 0025 mse= 0.053444982


In [233]:
def calc(x, y):
    # Returns predictions and error
    predictions = neural_net(x)
    error = tf.reduce_mean(tf.square(y - predictions))
    return [ predictions, error ]

In [234]:
test_cost = calc(X_test, y_test)[1]

In [235]:
with tf.Session() as sess:
    sess.run(init)
    print('Cost (MSE) for test data: {0}'.format(sess.run(test_cost)))

Cost (MSE) for test data: 0.06500223278999329
